Issue suivie : https://github.com/CashStory/training-datamining-mds/issues/2



*** Imports :**

In [0]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import plotly as plt
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots

*** Variables :**

In [0]:
# URLs
base_url_entree = 'https://geodes.santepubliquefrance.fr/GC_indic.php?lang=fr&prodhash=3c0e7522&indic=incid_rea&dataset=covid_hospit_incid&view=map2&filters=jour='
base_url_total =' https://geodes.santepubliquefrance.fr/GC_indic.php?lang=fr&prodhash=3c0e7522&indic=rea&dataset=covid_hospit&view=map2&filters=sexe=0,jour='

# Liste des départements
DEPARTMENTS = ['Ain', 'Aisne', 'Allier', 'Alpes-de-Haute-Provence', 'Hautes-Alpes', 'Alpes-Maritimes', 'Ardèche', 'Ardennes', 'Ariège', 'Aube', 'Aude', 'Aveyron',
               'Bouches-du-Rhône', 'Calvados', 'Cantal', 'Charente', 'Charente-Maritime', 'Cher', 'Corrèze', 'Côte-d\'Or', 'Côtes-d\'Armor', 'Creuse', 'Dordogne',
               'Doubs', 'Drôme', 'Eure', 'Eure-et-Loir', 'Finistère', 'Corse-du-Sud', 'Haute-Corse', 'Gard','Haute-Garonne','Gers','Gironde','Hérault',
               'Ille-et-Vilaine','Indre','Indre-et-Loire','Isère','Jura','Landes','Loir-et-Cher','Loire','Haute-Loire','Loire-Atlantique','Loiret','Lot',
               'Lot-et-Garonne','Lozère','Maine-et-Loire','Manche','Marne','Haute-Marne','Mayenne','Meurthe-et-Moselle','Meuse','Morbihan','Moselle','Nièvre',
               'Nord','Oise','Orne','Pas-de-Calais','Puy-de-Dôme','Pyrénées-Atlantiques','Hautes-Pyrénées','Pyrénées-Orientales','Bas-Rhin','Haut-Rhin','Rhône',
               'Haute-Saône','Saône-et-Loire','Sarthe','Savoie','Haute-Savoie','Paris','Seine-Maritime','Seine-et-Marne','Yvelines','Deux-Sèvres','Somme','Tarn',
               'Tarn-et-Garonne','Var','Vaucluse','Vendée','Vienne','Haute-Vienne','Vosges','Yonne','Territoire de Belfort','Essonne','Hauts-de-Seine',
               'Seine-Saint-Denis','Val-de-Marne','Val-d\'Oise','Guadeloupe','Martinique','Guyane','La Réunion','Mayotte']

# Nombre de jours
lissage_jours = 30

# Pour contenir les x derniers jours, x étant la variable "lissage_jours"
DATES = []

# Les indices contiennent x tableaux ordonnés en fonction de "DATES" contenant les données des départements ordonné comme "DEPARTEMENTS"
INDICES_TEMP_ENTREES = []
INDICES_TEMP_REANIMATION = []
INDICES_TEMP_COURANT = []
INDICES_ENTREES = []
INDICES_COURANT = []
INDICES_SORTIES = []

*** Récupération des données :**

In [0]:
for i in range(lissage_jours + 1):
  # Génération des dates
  day = (datetime.today() - timedelta(days = (lissage_jours - i))).isoformat().split("T")[0]
  DATES.append(day)

  # Récupération des entrées en réanimation
  url = (base_url_entree + day)
  response = requests.get(url)
  json = response.json()
  INDICES_TEMP_ENTREES.append(json['content']['distribution']['values'])
  for value in json['content']['distribution']['values']:
    INDICES_ENTREES.append(value)

  # Récupération des personnes actuellement en réanimation
  url = (base_url_total + day)
  response = requests.get(url)
  json = response.json()
  INDICES_TEMP_REANIMATION.append(json['content']['distribution']['values'])

*** Calcul des données :**

In [0]:
for i in range(1, lissage_jours + 1):
  for j in range(len(DEPARTMENTS)):
    INDICES_TEMP_COURANT.append([])
    INDICES_TEMP_COURANT[i-1].append(INDICES_TEMP_REANIMATION[i][j] - INDICES_TEMP_REANIMATION[i - 1][j])
    INDICES_SORTIES.append(INDICES_TEMP_ENTREES[i][j] - INDICES_TEMP_COURANT[i - 1][j])

INDICES_ENTREES = INDICES_ENTREES[len(DEPARTMENTS) - 1 : len(INDICES_ENTREES) - 1]
DATES.pop(0)

for value in INDICES_TEMP_COURANT:
  for v in value:
    INDICES_COURANT.append(v)

*** Mise en forme des données :**

In [0]:
iterables  = []
iterables.append(DATES)
iterables.append(DEPARTMENTS)
idx = pd.MultiIndex.from_product(iterables, names=['DATE', 'ZONE'])

datas = []
for i in range(len(iterables[1]) * len(iterables[0])) :
  datas.append(np.array([INDICES_ENTREES[i], INDICES_SORTIES[i],INDICES_COURANT[i]]))

df = pd.DataFrame(datas, index=idx, columns=['ENTREES', 'SORTIES', 'SOLDES'])
df

ENTREES  SORTIES  SOLDES
DATE       ZONE                                             
2020-03-22 Ain                            0        0       1
           Aisne                          1        0       1
           Allier                         1        0       0
           Alpes-de-Haute-Provence        0        0       0
           Hautes-Alpes                   0        0       0
...                                     ...      ...     ...
2020-04-20 Guadeloupe                     4        0       0
           Martinique                     0        0       0
           Guyane                         0        0       0
           La Réunion                     0        0       0
           Mayotte                        0        0       0

[3030 rows x 3 columns]

*** Plotting :**
- [ ] Ajouter filtre département
- [x] Agrandir graphique (enjoliver)
- [x] Séparer graphique Entree/Sortie et Solde
- [x] Mettre graphique en mode "area"
- [ ] (Optionnel) Mettre le nombre de jour souhaité
- [ ] (Optionnel) Séparer décès/sortie hopital de sorties

In [0]:
#df.plot();
#fig = make_subplots(
#     rows=2, cols=1,
#     column_widths=[1],
#     row_heights=[0.6, 0.4],
#     specs=[[ None, {"type": "bar"}],
#            [None, {"type": "surface"}]])


fig = go.Figure()
df = df.groupby(level=0).sum().reset_index()
fig.add_trace(go.Scatter(x=df.DATE, y=df.ENTREES, fill='tozeroy',name="Entrees",line=dict(width=0.5,color="rgb(160,0,0)"),line_shape='spline')) # fill down to xaxis
fig.add_trace(go.Scatter(x=df.DATE, y=df.SORTIES, fill='tozeroy',name="Sorties",line=dict(width=0.5,color="rgb(0,160,0)"),line_shape='spline')) # fill to trace0 y
fig.update_layout(width=1500, height=520, plot_bgcolor='rgb(255,255,255)')

fig.show()